# Vanilla Pytorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# Transform for the MNIST data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalizing the dataset
])

# Loading the training data for MNIST
train_set = MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)

# Define a simple 4-layer neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.layer_stack = nn.Sequential(
            nn.Linear(28*28, 128),# dense layer with no activation
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)  # 10 classes for MNIST digits
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layer_stack(x)
        return logits

# Initialize the network
model = SimpleNN()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Function to train the network
def train_network(train_loader, model, criterion, optimizer, epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward() # this is where the model learns by backpropagating
            optimizer.step() # this is where the model updates its weights

            running_loss += loss.item()
            if i % 100 == 99:  # print every 100 mini-batches
                print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

    print('Finished Training')

# Training the model
train_network(train_loader, model, criterion, optimizer, epochs=5)


100%|██████████| 9912422/9912422 [00:01<00:00, 5206453.32it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:02<00:00, 776985.35it/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch: 1, Batch: 100, Loss: 1.9742
Epoch: 1, Batch: 200, Loss: 0.7044
Epoch: 1, Batch: 300, Loss: 0.4767
Epoch: 1, Batch: 400, Loss: 0.4055
Epoch: 1, Batch: 500, Loss: 0.3639
Epoch: 1, Batch: 600, Loss: 0.3266
Epoch: 1, Batch: 700, Loss: 0.2714
Epoch: 1, Batch: 800, Loss: 0.2703
Epoch: 1, Batch: 900, Loss: 0.2613
Epoch: 2, Batch: 100, Loss: 0.2133
Epoch: 2, Batch: 200, Loss: 0.2187
Epoch: 2, Batch: 300, Loss: 0.1955
Epoch: 2, Batch: 400, Loss: 0.1916
Epoch: 2, Batch: 500, Loss: 0.1916
Epoch: 2, Batch: 600, Loss: 0.1770
Epoch: 2, Batch: 700, Loss: 0.1554
Epoch: 2, Batch: 800, Loss: 0.1507
Epoch: 2, Batch: 900, Loss: 0.1606
Epoch: 3, Batch: 100, Loss: 0.1449
Epoch: 3, Batch: 200, Loss: 0.1449
Epoch: 3, Batch: 300, Loss: 0.1466
Epoch: 3, Batch: 400, Loss: 0.1247
Epoch: 3, Batch: 500, Loss: 0.1271
Epoch: 3, Batch: 600, Loss: 0.1435
Epoch: 3, Batch: 700, Loss: 0.1326
Epoch: 3, Batch: 800, Loss: 0.1368
Epoch: 3, Batch

# using pytorch lightning

In [7]:
!pip install --upgrade pytorch-lightning


  Obtaining dependency information for pytorch-lightning from https://files.pythonhosted.org/packages/56/ed/192d7518b15a06452f480346eeebe1d1d4595af80687e142b2e6f18539fd/pytorch_lightning-2.2.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/801.6 kB ? eta -:--:--
   - ------------------------------------- 30.7/801.6 kB 660.6 kB/s eta 0:00:02
   - ------------------------------------- 30.7/801.6 kB 660.6 kB/s eta 0:00:02
   --- ----------------------------------- 81.9/801.6 kB 919.0 kB/s eta 0:00:01
   -------- ----------------------------- 174.1/801.6 kB 952.6 kB/s eta 0:00:01
   ---------- --------------------------- 225.3/801.6 kB 981.9 kB/s eta 0:00:01
   ---------------- ----------------------- 337.9/801.6 kB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 491.5/801.6 kB 1.5 MB/s eta 0:00:01
   ------------------------------- -------- 624.6/801.6 kB 1.7 MB/s eta 0:00:01
   -------------------------------------- - 778.2/801.6 kB 1.9 MB/s et

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
# from pytorch_lightning.metrics.functional import accuracy

# import accurayc from torchmetrics
from torchmetrics.functional import accuracy

# if you want to test out something, use keras, for something small or use pytorch for something big

class MNISTClassifier(pl.LightningModule):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1) # 1 input channel, 32 output channels, 3x3 kernel, stride of 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128) # self.fc1 = nn.Linear(linear_unit_1, output_unit)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2)(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        acc = accuracy(y_hat, y, task='MULTICLASS', num_classes = 10)
        # self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        # self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        acc = accuracy(y_hat, y, task='MULTICLASS', num_classes = 10)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)
# Define data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
mnist_train = MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = MNIST(root='./data', train=False, download=True, transform=transform)

# Split train dataset into train and validation sets
mnist_train, mnist_val = random_split(mnist_train, [55000, 5000])

# Define dataloaders
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=64)
test_loader = DataLoader(mnist_test, batch_size=64)

# Initialize model
model = MNISTClassifier()

# Initialize PyTorch Lightning trainer with corrected GPU argument
trainer = pl.Trainer(
    max_epochs=1,
    callbacks=[
        pl.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        pl.callbacks.ModelCheckpoint(monitor='val_loss', filename='best_model', save_top_k=1, mode='min')
    ]
)
# Train the model
trainer.fit(model, train_loader, val_loader)

# Test the model
# trainer.test(test_dataloaders=test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type      | Params
---------------------------------------
0 | conv1    | Conv2d    | 320   
1 | conv2    | Conv2d    | 18.5 K
2 | dropout1 | Dropout2d | 0     
3 | dropout2 | Dropout2d | 0     
4 | fc1      | Linear    | 1.2 M 
5 | fc2      | Linear    | 1.3 K 
---------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.800     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


# Hydra

In [21]:
!pip install hydra-core


  Obtaining dependency information for hydra-core from https://files.pythonhosted.org/packages/c6/50/e0edd38dcd63fb26a8547f13d28f7a008bc4a3fd4eb4ff030673f22ad41a/hydra_core-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for omegaconf<2.4,>=2.2 from https://files.pythonhosted.org/packages/e3/94/1843518e420fa3ed6919835845df698c7e27e183cb997394e4a670973a65/omegaconf-2.3.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/117.0 kB ? eta -:--:--
     --- ------------------------------------ 10.2/117.0 kB ? eta -:--:--
     --------- --------------------------- 30.7/117.0 kB 325.1 kB/s eta 0:00:01
     ------------------- ----------------- 61.4/117.0 kB 465.5 kB/s eta 0:00:01
     ------------------------------------ 117.0/117.0 kB 680.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/154.5 kB ? eta -:--:--
   -------------------

In [1]:
import hydra
from hydra import compose, initialize
from omegaconf import OmegaConf

# Initialize Hydra and compose the configuration
initialize(config_path="config", job_name="test_app")
cfg = compose(config_name="some_config")

# Print the entire config to verify it's loaded
print(OmegaConf.to_yaml(cfg))


C:\Users\Shivam\AppData\Local\Temp\ipykernel_47800\960703271.py:6: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="config", job_name="test_app")


db:
  driver: mysql
  user: omry
  pass: secret
secret_key: 123456
neural_network:
  layer_1:
    units: 64
    activation: relu
  layer_2:
    units: 32
    activation: relu



In [2]:
cfg.neural_network.layer_1.units

# use secret key in BOTO3

64

In [3]:
type(cfg.neural_network.layer_1.units)

int

: 

# Pyroot utils

In [ ]:
# import libraries
import sys
import pyrootutils
root = pyrootutils.setup_root(sys.path[0], pythonpath=True, cwd=True)

#setup the path of the module to the parent directory

link to templates

https://github.com/ashleve/lightning-hydra-template

In [4]:
from scipy.stats import norm


norm.ppf(0.95)

1.6448536269514722

: 